***Challenge 1***

Here the goal is to train on 25 samples. In this preliminary testbed the evaluation will be done on a 2000 sample validation set. Note in the end the final evaluation will be done on the full CIFAR-10 test set as well as potentially a separate dataset. The validation samples here should not be used for training in any way, the final evaluation will provide only random samples of 25 from a datasource that is not the CIFAR-10 training data.

Feel free to modify this testbed to your liking, including the normalization transformations etc. Note however the final evaluation testbed will have a rigid set of components where you will need to place your answer. The only constraint is the data. Refer to the full project instructions for more information.


Setup training functions. Again you are free to fully modify this testbed in your prototyping within the constraints of the data used. You can use tools outside of pytorch for training models if desired as well although the torchvision dataloaders will still be useful for interacting with the cifar-10 dataset.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load and prepare CIFAR-10 data only once
def load_cifar10_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    return train_dataset

# Prepare a test dataset (assuming classes not seen during training)
def load_cifar10_test_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    return test_dataset

# Custom dataset class
class CustomCIFAR10(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

def sample_task(dataset, num_samples_per_class=5, used_indices=None):
    if used_indices is None:
        used_indices = set()  # Use an empty set if none provided

    chosen_classes = random.sample(range(10), 2)  # Select two random classes
    class_map = {chosen_classes[i]: i for i in range(2)}
    data = []
    labels = []
    indices_per_class = {class_map[chosen_classes[0]]: [], class_map[chosen_classes[1]]: []}

    for idx, (image, label) in enumerate(dataset):
        if label in chosen_classes and idx not in used_indices:
            class_label = class_map[label]
            if len(indices_per_class[class_label]) < num_samples_per_class:
                data.append(image)
                labels.append(class_label)
                indices_per_class[class_label].append(idx)
                used_indices.add(idx)
            # Correct checking of completion for all class indices
            if all(len(indices_per_class[c]) == num_samples_per_class for c in indices_per_class):
                break

    return CustomCIFAR10(data, labels)


In [ ]:
def train(model, dataset, device, epochs=50, tasks_per_epoch=5, num_samples_per_class=5):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        used_indices = set()
        for _ in range(tasks_per_epoch):
            # Ensure the correct order and usage of arguments when calling sample_task
            task_dataset = sample_task(dataset, num_samples_per_class=num_samples_per_class, used_indices=used_indices)
            task_loader = DataLoader(task_dataset, batch_size=10, shuffle=True)

            for data, targets in task_loader:
                data, targets = data.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(data)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


def test(model, dataset, device, num_classes=2, num_samples=5, n_inner_iter=5, inner_lr=0.01):
    accuracies = []
    for _ in range(10):  # Run several test tasks for better statistical measure
        # Ensure test classes are either new or correctly partitioned
        task_dataset = sample_task(dataset, num_samples_per_class=num_samples)
        task_loader = DataLoader(task_dataset, batch_size=num_samples, shuffle=True)

        # Clone and adapt model to new task
        adapted_model = type(model)().to(device)
        adapted_model.load_state_dict(model.state_dict())
        optimizer = optim.SGD(adapted_model.parameters(), lr=inner_lr)

        # Adaptation phase: Fine-tune on the new task
        for data, targets in task_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = adapted_model(data)
            loss = nn.CrossEntropyLoss()(outputs, targets)
            loss.backward()
            optimizer.step()

        # Evaluation phase: Test the fine-tuned model
        correct = 0
        total = 0
        with torch.no_grad():
            for data, targets in task_loader:
                data, targets = data.to(device), targets.to(device)
                outputs = adapted_model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        accuracy = correct / total
        accuracies.append(accuracy)

    average_accuracy = np.mean(accuracies)
    print(f'Average Test Accuracy on new tasks: {average_accuracy * 100:.2f}%')
    return average_accuracy



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18


class Net(nn.Module):
    def __init__(self, num_classes=2, dropout_rate=0.5):
        super(Net, self).__init__()
        self.resnet = resnet18(pretrained=False)
        num_features = self.resnet.fc.in_features

        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

The below tries  2 random problem instances. In your development you may choose to prototype with 1 problem instances but keep in mind for small sample problems the variance is high so continously evaluating on several subsets will be important.

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
import time


accs = []
times = []


for seed in range(1, 5):
  train_dataset = load_cifar10_data()
  val_dataset = load_cifar10_test_data()

  model = Net()
  model.to(device)

  start_time = time.time()
  train(model, train_dataset, device, 100)
  end_time = time.time()

  times.append(end_time - start_time)

  accuracy = test(model, val_dataset, device, num_classes=10, num_samples=200)
  accs.append(accuracy)

times = np.array(times)
accs = np.array(accs)
print('Acc over 5 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
print(f"Average Time over 5 instances: {times.mean()} +-{times.std()}")


100%|██████████| 170498071/170498071 [00:03<00:00, 43568124.12it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1, Loss: 1.5389602184295654
Epoch 2, Loss: 2.4616198539733887
Epoch 3, Loss: 1.4926371574401855
Epoch 4, Loss: 1.2508658170700073
Epoch 5, Loss: 1.007866621017456
Epoch 6, Loss: 0.6581655740737915
Epoch 7, Loss: 0.8955932855606079
Epoch 8, Loss: 0.7133429646492004
Epoch 9, Loss: 0.6460853815078735
Epoch 10, Loss: 0.42995983362197876
Epoch 11, Loss: 1.0760440826416016
Epoch 12, Loss: 0.5978276133537292
Epoch 13, Loss: 1.0797353982925415
Epoch 14, Loss: 0.8105818033218384
Epoch 15, Loss: 1.537963628768921
Epoch 16, Loss: 0.5435749888420105
Epoch 17, Loss: 0.6970846652984619
Epoch 18, Loss: 0.48743557929992676
Epoch 19, Loss: 0.8558869361877441
Epoch 20, Loss: 0.8421810269355774
Epoch 21, Loss: 0.7230194807052612
Epoch 22, Loss: 0.4385087490081787
Epoch 23, Loss: 0.7554768323898315
Epoch 24, Loss: 0.7635484933853149
Epoch 25, Loss: 1.163048505783081
Epoch 26, Loss: 0.9945308566093445
Epoch 27, Loss: 0.6017218828201294
Epoch 28, Loss: 0.8327981233596802
Epoch 29, Loss: 1.287109255790